In [1]:
import sqlite3
import RPi.GPIO as GPIO
import datetime

In [2]:
relay_ch = 22

GPIO.setwarnings(True)
GPIO.setmode(GPIO.BCM)
GPIO.setup(relay_ch, GPIO.OUT)

/tmp/ipykernel_15861/4050114825.py:5: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(relay_ch, GPIO.OUT)


In [3]:
connection = sqlite3.connect('solarthermal.db')
cursor = connection.cursor()

In [5]:
query = 'SELECT timestamp FROM solarthermal WHERE pump=1 ORDER BY timestamp DESC LIMIT 1'
cursor.execute(query)
last_time_on = cursor.fetchone()[0]
last_time_on = datetime.datetime.strptime(last_time_on, '%Y-%m-%d %H:%M:%S.%f')
seconds_since_shutoff = (datetime.datetime.now()-last_time_on).total_seconds()
seconds_since_shutoff

85.614019

In [6]:
query = 'SELECT power, temp3 FROM solarthermal ORDER BY timestamp DESC LIMIT 1'
cursor.execute(query)
res = cursor.fetchone()
power, temp3 = res

In [7]:
def turn_off():
    GPIO.output(relay_ch, GPIO.LOW)
    
def turn_on():
    GPIO.output(relay_ch, GPIO.HIGH)

In [10]:
power = -1

In [11]:
now = datetime.datetime.now()

if power < 0:
    turn_off()
    insert_state({'timestamp': now, 'working': False, 'reason': 'negative power'})
elif temp3 < 5:
    turn_off()
    insert_state({ 'timestamp': now, 'working': False, 'reason': 'ambient temp'})

In [19]:
query = 'SELECT timestamp, working FROM controller ORDER BY timestamp DESC LIMIT 1'
cursor.execute(query)
timestamp, working = cursor.fetchone()
timestamp = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')

In [ ]:
query = 'SELECT timestamp FROM controller ORDER BY timestamp DESC LIMIT 1'
cursor.execute(query)
timestamp, working = cursor.fetchone()
timestamp = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')

11.973459

In [15]:
now = datetime.datetime.now()

if working == 0 and seconds_since_shutoff > 10:
    turn_on()
    insert_state({'timestamp': now, 'working': True, 'reason': 'Waiting period expired'})

In [16]:
connection.close()